In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [22]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [3]:
feed_temp_ins4 = log_rawDF.query('Time >= 1635246000 and Time <= 1641567595 and Unit =="FPC24" and Type == 2 and Parameter == "Reactor"')

In [4]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Recipe', regex=False, case=False, na=False)]

In [15]:
programID_DF.head(20)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350408,2021-11-2 8:37:25,1635838645,243296,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.0857142857142857
350510,2021-11-2 10:57:24,1635847044,749185,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.514285714285714
350657,2021-11-2 11:33:29,1635849209,742078,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.257142857142857
352123,2021-11-3 8:33:30,1635924810,738753,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.114285714285714
352176,2021-11-3 10:17:39,1635931059,739134,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.685714285714286
352234,2021-11-3 10:29:26,1635931766,741717,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.342857142857143
352821,2021-11-3 15:49:45,1635950985,246696,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.142857142857143
352842,2021-11-3 16:8:45,1635952125,738636,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.857142857142857
352891,2021-11-3 16:20:18,1635952818,238720,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.428571428571429
354615,2021-11-5 10:51:40,1636105900,248642,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.142857142857143


In [6]:
new_DF = log_rawDF.copy()

In [7]:
new_DF_ins4 = new_DF.query('Time >= 1635246000 and Time <= 1641567595')

In [8]:
new_ins4 = new_DF_ins4[new_DF_ins4['Message'].str.contains('FPC24', regex=False, case=False, na=False)]

In [9]:
new_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,"Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.09083</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03131</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07249</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.02936</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,"Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.25719</td><td width=""12.5%"">level</td><td width=""12.5%"">0.28841</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.25714,0.00000


In [10]:
no_html_DF =  new_ins4.copy()

In [11]:
#no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<[^<>]*>', '', regex=True)
no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)

In [14]:
no_html_DF.head(20)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose),0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose),0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose),0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose),0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose),0.25714,0.00000
350669,2021-11-2 11:41:2,1635849662,356071,2,FFU,T10,Dose ended at low level for FeedWaterAcid to Tank = FPC24_Reactor (DB total dose and current real dose),0.21921,0.25043
350670,2021-11-2 11:41:10,1635849670,878061,2,FFU,T10,Dose ended at low level for FeedWaterAcid to Tank = FPC24_Reactor (DB total dose and current real dose),0.25728,0.25488
352136,2021-11-3 8:37:15,1635925035,351895,2,FFU,T10,Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose),0.11429,0.00000
352172,2021-11-3 10:16:34,1635930994,365017,2,FFU,T10,Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose),0.12225,0.07131
352213,2021-11-3 10:19:50,1635931190,841621,2,FFU,T10,Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose),0.68571,0.00000


In [13]:
no_html_DF.tail()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
446599,2021-12-29 16:23:29,1640791409,196243,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),0.22822,0.21332
446622,2021-12-29 16:30:7,1640791807,667000,2,FFU,T10,Dose started for NaOH_0.1M_rinse to Tank = FPC24_Reactor (target dose and current real dose),0.91429,0.00000
446634,2021-12-29 16:40:9,1640792409,187728,2,FFU,T10,Dose ended at low level for NaOH_0.1M_rinse to Tank = FPC24_Reactor (DB total dose and current real dose),0.91182,0.90214
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose),1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),1.14025,1.14535


In [ ]:
#and Unit =="FPC24" and Type == 2 and Parameter == "Reactor"

In [58]:
tmp1 = log_rawDF.query('Time >= 1635847985 and Time <= 1635849209 and Type == 2')

In [59]:
tmp1.head(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350590,2021-11-2 11:13:5,1635847985,370133,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC14_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07108</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03775</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.67507,0.59843
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350592,2021-11-2 11:13:5,1635847985,877172,2,FFU,feeding,pHCorr product changed (old and new),not found,Complete 3_5de half
350593,2021-11-2 11:13:5,1635847985,880759,2,FFU,feeding,pHCorr product not found in settings DB =&gt; reset to not found,NaN,NaN
350594,2021-11-2 11:13:6,1635847986,387688,2,FFU,feeding,pHCorr product changed (old and new),Complete 3_5de half,not found
350595,2021-11-2 11:13:7,1635847987,839229,2,FFU,T10,Recipe of Complete 3_5de half started (kg),NaN,0.514285714285714
350596,2021-11-2 11:13:8,1635847988,342968,2,FFU,T10,New batch number generated,FFU_T10_20211102_105727,FFU_T10_20211102_111308
350597,2021-11-2 11:13:9,1635847989,856196,2,FFU,T71,"Dose started for CompWater to Tank = FFU_T10 (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.42573</td><td width=""12.5%"">level</td><td width=""12.5%"">1.42581</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.35362,0.00000
350598,2021-11-2 11:13:11,1635847991,856785,2,FFU,T64,"Dose started for NaNO3_2.47M to Tank = FFU_T10 (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.03151</td><td width=""12.5%"">level</td><td width=""12.5%"">1.03192</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00859,0.00000
350600,2021-11-2 11:13:13,1635847993,843668,2,FFU,T42,"Dose started for NaCl_4,3M to Tank = FFU_T10 (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.75024</td><td width=""12.5%"">level</td><td width=""12.5%"">0.75022</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08143,0.00000
